In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-google-genai langgraph_sdk

In [ ]:
from IPython.display import display, Image

from typing import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt
from langgraph.graph import StateGraph, START, END

In [ ]:
class State(TypedDict):
    input: str

def step_1(state: State) -> State:
    print("---Step 1---")
    return state

def step_2(state: State) -> State:

    if len(state["input"]) > 5:
        raise interrupt(f"Received input longer than 5 characters: {state['input']}")

    print("---Step 2---")
    return state

def step_3(state: State) -> State:
    print("---Step 3---")
    return state

builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.add_edge(START, "step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

display(Image(graph.get_graph().draw_mermaid_png()))



In [ ]:
initial_state = {"input": "Hello, World!"}
thread_config = {"configurable": {"thread_id": "1"}}

for event in graph.stream(initial_state, thread_config, stream_mode="values"):
    print(event)

In [ ]:
state = graph.get_state(thread_config)
print(state.next)

In [ ]:
print(state.tasks)

In [ ]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)

In [ ]:
state = graph.get_state(thread_config)
print(state.next)

In [ ]:
graph.update_state(
    thread_config,
    {"input": "Hi!"}
)   

In [ ]:
for event in graph.stream(None, thread_config, stream_mode="values"):
    print(event)